In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

from itertools import product

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report

In [67]:
def seasonal_return(data, symbol, start_date, end_date, first_year, last_year):
    data_list = []
    # Deal with Feb 29: assign start/end dates to Mar 1
    if start_date == '02-29': start_date = '03-01'
    if end_date == '02-29': end_date = '03-01'
    for year in range(first_year, (last_year + 1)):
        full_start_date = str(year) + '-' + start_date
        full_end_date = str(year) + '-' + end_date
        trade_start_date = data.loc[full_start_date, 'Price Date']
        trade_end_date = data.loc[full_end_date, 'Price Date']
        start_price = data.loc[full_start_date, symbol]

        # If price data is missing, skip that year
        if np.isnan(start_price):
            continue
        end_price = data.loc[full_end_date, symbol]
        if np.isnan(end_price):
            continue
        returns = np.log(end_price / start_price)
        data_list.append([symbol, year, trade_start_date, start_price,
                          trade_end_date,
                          end_price, returns])

    df = pd.DataFrame(data_list, columns=['Symbol', 'Year', 'Init Date',
                                          'Init Price', 'Final Date', 'Final Price', 'Return'])
    return df


def return_stats(x, risk_free_rate=0):
    d = {}
    d['N'] = x['Symbol'].count()
    d['avg r'] = x['Return'].mean()
    d['vol'] = x['Return'].std()
    downsides = x[x['Return'] < risk_free_rate]['Return']
    d['downside dev'] = 0 if downsides.count() == 0 else downsides.std()
    upsides = x[-x['Return'] < risk_free_rate]['Return']
    d['upside dev'] = 0 if upsides.count() == 0 else upsides.std()
    d['up'] = sum(x['Return'] > risk_free_rate)
    return pd.Series(d, index=['N', 'avg r', 'vol', 'downside dev',
                               'upside dev', 'up'])


# Historical rate of stock going up (i.e. winrate if we are long)
def historical_up_rate(data, symbol, start_date, end_date, first_year,
                       last_year):
    up_list = []
    # Deal with Feb 29: assign start/end dates to Mar 1
    if start_date == '02-29': start_date = '03-01'
    if end_date == '02-29': end_date = '03-01'
    for year in range(first_year, (last_year + 1)):
        full_start_date = str(year) + '-' + start_date
        full_end_date = str(year) + '-' + end_date
        trade_start_date = data.loc[full_start_date, 'Price Date']
        trade_end_date = data.loc[full_end_date, 'Price Date']
        start_price = data.loc[full_start_date, symbol]

        # If price data is missing, skip that year
        if np.isnan(start_price):
            continue
        end_price = data.loc[full_end_date, symbol]
        if np.isnan(end_price):
            continue

        if end_price >= start_price:
            ret = 1
        else:
            ret = 0
        up_list.append(ret)
    return np.mean(up_list)


# Identify long and short positions using the same seasonal strategy/criteria,
# using the 10-year seasonal lookback, but conducting the trades over
# 2019-2021. This means we look at seasonal activity going back to 2009.

# Pull the adjusted close prices off Yahoo Finance
df = pd.read_csv("../files/S&P500-Symbols.csv")
tickers = list(df['Symbol'])
start_date = '1989-01-01'
end_date = '2024-03-24'  # Get data a few days past end of year to backfill

# Either pull from Yahoo finance, or for read the pre-downloaded CSV
data = pd.DataFrame(yf.download(tickers, start_date, end_date)['Adj Close'])


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1989-01-01 -> 2024-03-24)')


KeyError: 'Date'

In [71]:
data = data.reset_index()
# data.reset_index().to_csv("files/S&P500-adjusted-close.csv", index=False)
#data = pd.read_csv('../files/S&P500-adjusted-close.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
all_dates = pd.date_range(start_date, end_date)
data['Price Date'] = data.index

# Backfill with trading prices for missing dates
data = data.reindex(all_dates, method='bfill')
sp500_dates_added = pd.read_csv("../files/S&P500-Info.csv")[['Symbol', 'Date added']]
all_stocks = data.columns.drop(labels='Price Date')
data = data[data.index < '2024-03-24']

# Restrict to stocks that have data back in 2009, total of 432
sub_cols = data.columns[data.loc['2009-01-01'].notna()]
sub_cols = sub_cols.append(pd.Index(['Price Date']))
sub_stocks = data[sub_cols].columns.drop(labels='Price Date')
sub_data = data[sub_cols][data.index >= '2009-01-01']

# Specifications for the technical strategy
hold_range = [7, 14, 28]  # hold for 1, 2, or 4 weeks
delay_range = [0, 5, 10]  # search for trades 0, 5, 10 days after trade window

# Trade windows start on the 1st/15th of each month
start_months = list(range(1, 12 + 1))
start_days = ['-01', '-15']
initial_dates = [str(i) + j for i, j in product(start_months, start_days)]

In [73]:

# Import the CSV of all returns
all_returns = pd.read_csv('../files/seasonal_trades_2019_2021.csv')

# Annualize returns, get year of trade
all_returns['annualized r'] = (all_returns['avg r'] * 365 /
                               all_returns['hold length'])
all_returns['trade year'] = all_returns['trade window'].str.slice(0, 4)

# Part 2: Get feature data at the time of trade

# This time, our criteria is annualized returns of at least 40%, a 6/10+ winrate
# in the last 10 years, and total return of at least 2%. The choice of 2% is to
# select for technical trades that have a fairly "obvious" seasonal signal
# relative to the typical day-to-day volatility in stock prices (usually ~1%).
# From here, sort by highest Sharpe ratio using historical seasonal trades,
# and select the top 10 for each trade window

long_positions = all_returns[(all_returns['annualized r'] > 0.4) &
                             (all_returns.up >= 6) &
                             (all_returns['avg r'] > .02)].sort_values(
    'Sharpe Long', ascending=False).groupby('trade window').head(10)

# Define the feature functions

# Historical winrates
long_positions['past3yr'] = long_positions.apply(lambda row: historical_up_rate(
    data, row['Symbol'], row['start date'][5:10], row['end date'],
    int(row['trade year']) - 3, int(row['trade year']) - 1), axis=1)

long_positions['past5yr'] = long_positions.apply(lambda row: historical_up_rate(
    data, row['Symbol'], row['start date'][5:10], row['end date'],
    int(row['trade year']) - 5, int(row['trade year']) - 1), axis=1)

# Daily volatility with exponentially weighted moving average
# We get in a bit of trouble with the backfilled data: it would result in
# repetitions of the same price over non-trading days, reducing volatility,
# so move forward to the day we would trade and use non-backfilled data

data_no_backfill = data[data.index == data['Price Date']]


# Note that these functions are shifted by 1 day backwards, to avoid knowing the
# current day's closing price when making a trade decision during the day

def get_ewm_vol(data, symbol, date, span=10):
    stock_rets = np.log(data[symbol] / data[symbol].shift(1)).shift(1)
    return stock_rets.ewm(span).std().loc[date]


long_positions['ewm_vol'] = long_positions.apply(lambda row: get_ewm_vol(
    data_no_backfill, row['Symbol'], data.loc[row['start date']]['Price Date'],
    30), axis=1)


# EWMA of price: compare between a long/short window to determine
# whether the signal is long
def get_ewm_momentum(data, symbol, date, long_window, short_window):
    short_ewma = data[symbol].ewm(short_window).mean().shift(1).loc[date]
    long_ewma = data[symbol].ewm(long_window).mean().shift(1).loc[date]
    return np.log(short_ewma / long_ewma)


long_positions['long_momentum'] = long_positions.apply(
    lambda row: get_ewm_momentum(data_no_backfill, row['Symbol'],
                                 data.loc[row['start date']]['Price Date'],
                                 long_window=10, short_window=5), axis=1)


# Generates an X-day back return. With days_back = 1, it is yesterday's observed
# return

def get_recent_return(data, symbol, date, days_back=1):
    return np.log(data[symbol] / data[symbol].shift(days_back)).shift(1).loc[date]


long_positions['yesterday_ret'] = long_positions.apply(
    lambda row: get_recent_return(data_no_backfill, row['Symbol'],
                                  data.loc[row['start date']]['Price Date'],
                                  days_back=1), axis=1)

# Get SP500 EWMA and yesterday returns, making sure not to backfill
spx = pd.DataFrame(yf.download('^SPX', start_date, end_date)['Adj Close'])
all_dates = pd.date_range(start_date, end_date)

long_positions['sp500_ewm_vol'] = long_positions.apply(
    lambda row: get_ewm_vol(spx, 'Adj Close',
                            data.loc[row['start date']]['Price Date'], 10), axis=1)

long_positions['sp500_ewma'] = long_positions.apply(
    lambda row: get_ewm_momentum(spx, 'Adj Close',
                                 data.loc[row['start date']]['Price Date'],
                                 long_window=10, short_window=5), axis=1)

long_positions['sp500_yest_ret'] = long_positions.apply(
    lambda row: get_recent_return(spx, 'Adj Close',
                                  data.loc[row['start date']]['Price Date'],
                                  days_back=1), axis=1)


# Get the actual returns for the start/end date
def get_actual_return(data, symbol, start_date, end_date):
    return np.log(data[symbol].loc[end_date] / data[symbol].loc[start_date])


long_positions['actual_return'] = long_positions.apply(
    lambda row: get_actual_return(data, row['Symbol'], row['start date'],
                                  row['trade year'] + '-' + row['end date']), axis=1)

[*********************100%%**********************]  1 of 1 completed


In [74]:
later_returns = pd.read_csv('../files/seasonal_trades_2022_2023.csv')

later_long_positions = later_returns[(later_returns['annualized r'] > 0.4) &
                                     (later_returns.up >= 6) &
                                     later_returns['avg r'] > .02].sort_values(
    'Sharpe Long', ascending=False).groupby('trade window').head(10)

# Calculate the feature values
later_long_positions['past3yr'] = later_long_positions.apply(lambda row: historical_up_rate(
    data, row['Symbol'], row['start date'][5:10], row['end date'],
    int(row['trade year']) - 3, int(row['trade year']) - 1), axis=1)

later_long_positions['past5yr'] = later_long_positions.apply(lambda row: historical_up_rate(
    data, row['Symbol'], row['start date'][5:10], row['end date'],
    int(row['trade year']) - 5, int(row['trade year']) - 1), axis=1)

later_long_positions['ewm_vol'] = later_long_positions.apply(
    lambda row: get_ewm_vol(data, row['Symbol'], row['start date'], 30), axis=1)

later_long_positions['long_momentum'] = later_long_positions.apply(
    lambda row: get_ewm_momentum(data, row['Symbol'], row['start date'],
                                 long_window=10, short_window=5), axis=1)

later_long_positions['yesterday_ret'] = later_long_positions.apply(
    lambda row: get_recent_return(data_no_backfill, row['Symbol'],
                                  data.loc[row['start date']]['Price Date'],
                                  days_back=1), axis=1)

later_long_positions['sp500_ewm_vol'] = later_long_positions.apply(
    lambda row: get_ewm_vol(spx, 'Adj Close',
                            data.loc[row['start date']]['Price Date'], 10), axis=1)

later_long_positions['sp500_ewma'] = later_long_positions.apply(
    lambda row: get_ewm_momentum(spx, 'Adj Close',
                                 data.loc[row['start date']]['Price Date'], long_window=10,
                                 short_window=5), axis=1)

later_long_positions['sp500_yest_ret'] = later_long_positions.apply(
    lambda row: get_recent_return(spx, 'Adj Close',
                                  data.loc[row['start date']]['Price Date'], days_back=1), axis=1)

later_long_positions['actual_return'] = later_long_positions.apply(
    lambda row: get_actual_return(data, row['Symbol'], row['start date'],
                                  str(row['trade year']) + '-' + row['end date']), axis=1)

later_long_positions['outcome'] = np.where(
    later_long_positions['actual_return'] > 0.02, 1, 0)

In [87]:
long_positions_through_2023 = pd.concat([long_positions, later_long_positions])
long_positions_through_2023['outcome'] = np.where(
    long_positions_through_2023['actual_return'] > 0.02, 1, 0)

X_through_2023 = long_positions_through_2023[['past3yr', 'past5yr', 'ewm_vol',
                                    'long_momentum', 'yesterday_ret', 'sp500_ewm_vol', 'sp500_ewma',
                                    'sp500_yest_ret']].values
y_through_2023 = long_positions_through_2023['outcome'].values

model = RandomForestClassifier(random_state=np.random.seed(1234))
model.fit(X_through_2023,y_through_2023)

RandomForestClassifier()

In [86]:
long_positions_through_2023

array([0, 1, 0, ..., 1, 0, 0])

In [92]:
long_2024 = pd.read_csv('seasonal_trades_03-24.csv')

In [93]:
long_2024['past3yr'] = long_2024.apply(lambda row: historical_up_rate(
    data, row['Symbol'], '03-22', row['end_date'],
    2021, 2023), axis=1)

long_2024['past5yr'] = long_2024.apply(lambda row: historical_up_rate(
    data, row['Symbol'], '03-22', row['end_date'],
    2019, 2023), axis=1)

long_2024['ewm_vol'] = long_2024.apply(
    lambda row: get_ewm_vol(data, row['Symbol'], '2024-03-22', 30), axis=1)

long_2024['long_momentum'] = long_2024.apply(
    lambda row: get_ewm_momentum(data, row['Symbol'], '2024-03-22',
                                 long_window=10, short_window=5), axis=1)

long_2024['yesterday_ret'] = long_2024.apply(
    lambda row: get_recent_return(data_no_backfill, row['Symbol'],
                                  data.loc['2024-03-22']['Price Date'],
                                  days_back=1), axis=1)

long_2024['sp500_ewm_vol'] = long_2024.apply(
    lambda row: get_ewm_vol(spx, 'Adj Close',
                            data.loc['2024-03-22']['Price Date'], 10), axis=1)

long_2024['sp500_ewma'] = long_2024.apply(
    lambda row: get_ewm_momentum(spx, 'Adj Close',
                                 data.loc['2024-03-22']['Price Date'], long_window=10,
                                 short_window=5), axis=1)

long_2024['sp500_yest_ret'] = long_2024.apply(
    lambda row: get_recent_return(spx, 'Adj Close',
                                  data.loc['2024-03-22']['Price Date'], days_back=1), axis=1)

In [94]:
X_2024 = long_2024[['past3yr', 'past5yr', 'ewm_vol', 'long_momentum',
                    'yesterday_ret', 'sp500_ewm_vol', 'sp500_ewma',
                    'sp500_yest_ret']].values

rf_trade_2024_prob = model.predict_proba(X_2024)[:, 1]
rf_trade_2024 = model.predict(X_2024)

In [97]:
rf_trade_2024_prob.max()

0.64

In [96]:
rf_trade_2024_prob

array([0.48      , 0.3025    , 0.36333333, 0.44      , 0.39333333,
       0.44      , 0.265     , 0.43666667, 0.35833333, 0.43      ,
       0.2575    , 0.41      , 0.35      , 0.36      , 0.33      ,
       0.31      , 0.17      , 0.285     , 0.26      , 0.39      ,
       0.24      , 0.38      , 0.5       , 0.425     , 0.24      ,
       0.42      , 0.34      , 0.64      , 0.43      , 0.31166667,
       0.405     , 0.44      , 0.50666667])